Esse último notebook, é o qual eu crio de fato o modelo final que vai ser utilizado para a aplicação web.
Nele, eu concentro esforço para criar um ensemble entre os 3 melhores modelos, e assim chegar em um resultado com uma melhor métrica e tendo cuidado para não aumentar muito a complexidade do modelo.

In [58]:
import pandas as pd
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler
from scipy.sparse import csr_matrix
from sklearn.pipeline import make_pipeline

from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [59]:
df = pd.read_csv("fully_annoteded.csv").dropna(subset=['y'])

df['view_per_day'] = round(df['view_count'] / df['tempo_desde_pub'], 4)
df = df.drop(['tempo_desde_pub'], axis=1)
df.head()

,title,y,upload_date,view_count,view_per_day
0,"""Kaggle: command not found fix"" for Mac Users",0,30/11/2020,216,0.7855
1,#1 - Data Science na PrÃ¡tica - Conhecendo o K...,0,03/08/2020,808,2.0508
2,"#AndroidDevChallenge - Helpful innovation, pow...",0,22/06/2020,5779436,13255.5872
3,#Data Science SQL Full Outer join Keyword | Ek...,0,28/08/2021,1,0.2500
4,#Data Science SQL Left join Keyword | Ekasclou...,0,28/08/2021,0,0.0000


### Random Forest Classifier

In [60]:
X = df.copy().drop(['y', 'upload_date'], axis=1)
y = df['y']

In [61]:
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.55, random_state=0)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((531, 3), (651, 3), (531,), (651,))

In [62]:
title_train = Xtrain['title']
title_val = Xval['title']

#min_df = minimo de exemplos de palavra
#ngram_range = numero de unigramas
title_vec = TfidfVectorizer(min_df=1, ngram_range=(1,1))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [63]:
Xtrain_wtitle = hstack([Xtrain.drop(['title'], axis=1), title_bow_train])
Xval_wtitle = hstack([Xval.drop(['title'], axis=1), title_bow_val])

Xtrain_wtitle.shape, Xval_wtitle.shape

((531, 1529), (651, 1529))

In [64]:
mdl = RandomForestClassifier(n_estimators=100, random_state=0, min_samples_leaf=1, class_weight='balanced', n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_jobs=6, random_state=0)

In [65]:
p = mdl.predict_proba(Xval_wtitle)[ : , 1]
print('precision: {} and roc: {}'.format(average_precision_score(yval, p), roc_auc_score(yval, p)))

precision: 0.32113680450630394 and roc: 0.7318334350213545


### LGBM (com os melhores parâmetros)

In [66]:
lr = 0.07473498545892235
max_depth = 9
min_child_samples = 1
subsample = 0.8020499855076734
colsample_bytree = 0.10705426389117481
n_estimators = 562
min_df = 1
ngram_range = 2

In [67]:
mdl2 = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth,
                      min_child_samples=min_child_samples, subsample=subsample,
                      colsample_bytree=colsample_bytree, bagging_freq=1, n_estimators=n_estimators, random_state=0,
                      class_weight="balanced", n_jobs=6)

mdl2.fit(Xtrain_wtitle, ytrain)

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


LGBMClassifier(bagging_freq=1, class_weight='balanced',
               colsample_bytree=0.10705426389117481,
               learning_rate=0.07473498545892235, max_depth=9,
               min_child_samples=1, n_estimators=562, n_jobs=6, num_leaves=512,
               random_state=0, subsample=0.8020499855076734)

In [68]:
p2 = mdl2.predict_proba(Xval_wtitle)[ : , 1]

D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [69]:
print('precision: {} and roc: {}'.format(average_precision_score(yval, p2), roc_auc_score(yval, p2)))

precision: 0.21154151038689378 and roc: 0.6838316046369737


### Logistic Regression (com os melhores parâmetros)

In [70]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

In [71]:
lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=1.5, n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('logisticregression',
                 LogisticRegression(C=1.5, n_jobs=6, random_state=0))])

In [72]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[: , 1]

In [73]:
print('precision: {} and roc: {}'.format(average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)))

precision: 0.28204431852326695 and roc: 0.7373398413666871


### Ensemble

Random Forest =       precision: 0.32113680450630394 and roc: 0.7318334350213545
LGBM =                precision: 0.21154151038689378 and roc: 0.6838316046369737
Logistic Regression = precision: 0.28204431852326695 and roc: 0.7373398413666871

Média Simples entre as previsões

In [74]:
p_ = (p + p2 + p_lr)
print('precision: {} and roc: {}'.format(average_precision_score(yval, p_), roc_auc_score(yval, p_)))

precision: 0.2729531626037601 and roc: 0.7363941427699817


Gerando correlação de pearson entre as previsões dos modelos, pra ver se elas sao parecidas

In [75]:
pd.DataFrame({"RF":p, "LGBM":p2, "LR":p_lr}).corr()

,RF,LGBM,LR
RF,1.000000,0.447122,0.752221
LGBM,0.447122,1.000000,0.533269
LR,0.752221,0.533269,1.000000


A célula abaixo foi a qual eu testei diversas combinações entre os algoritmos, com a finalidade de achar o melhor ensemble entre eles.

In [76]:
p_ = (0.9*p) + (0.1*p2)
print('precision: {} and roc: {}'.format(average_precision_score(yval, p_), roc_auc_score(yval, p_)))

precision: 0.3263577213482136 and roc: 0.7309945088468579


precision: 0.2803321904364961 and roc: 0.7274862721171446 -> (0.5*p) + (0.5*p2)
precision: 0.26942643781001285 and roc: 0.7266015863331299 -> (0.4*p) + (0.6*p2)
precision: 0.2887393059269871 and roc: 0.7281574130567419 -> (0.6*p) + (0.4*p2)
precision: 0.2985445675037923 and roc: 0.7290726052471018 -> (0.7*p) + (0.3*p2)
precision: 0.30362248570034883 and roc: 0.7292556436851738 -> (0.8*p) + (0.2*p2)
precision: 0.3263577213482136 and roc: 0.7309945088468579 -> (0.9*p) + (0.1*p2)

### Salvando os modelos

In [77]:
import joblib as jb

In [57]:
jb.dump(mdl, "random_forest_2021_09_19.pkl.z")
jb.dump(mdl2, "lgbm_2021_09_19.pkl.z")
jb.dump(lr_pipeline, "logistic_regression_2021_09_19.pkl.z")
jb.dump(title_vec, "vectorize_2021_09_19.pkl.z")

['logistic_regression_2021_09_19.pkl.z']